In [6]:
from scipy.sparse import csr_matrix, lil_matrix, load_npz, save_npz, hstack
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
import os

In [ ]:


path = '/projects/melloddy/data/y2run_final/data_prep/y2_ext/y2_ext_noaux/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [8]:
tuner_output_baseline_w_aux = '../aux_data_preperation/baseline_plus_aux_data/'
path = os.path.join(tuner_output_baseline_w_aux, 'matrices/cls/cls_T10_y.npz')
y_aux = load_npz(path)

In [3]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

In [ ]:
# Generate the y-sparse matrix for inference 
# Main tasks + fold 0 to limit the amount of predictions

In [9]:
path = os.path.join(tuner_output_baseline_w_aux, 'matrices/cls/cls_T11_fold_vector.npy')
folds = np.load(path)

In [11]:
path = os.path.join(tuner_output_baseline_w_aux, 'results_tmp/classification/T8c.csv')
t8c = pd.read_csv(path)

In [12]:
path = '../analysis/cp/summary_eps_0.05_mgd_cp.csv'
df_cp_mgd = pd.read_csv(path)

In [13]:
df_matching = pd.merge(
    df_cp_mgd
    ,t8c
    ,on=['input_assay_id','threshold']
    ,how='inner'
    ,suffixes=('_image','_baselineaux')
)

In [14]:
# main tasks 
# mind the difference between these two !
# cont_classification_task_id is the baseline + aux model 
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id'].tail())
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id_baseline'].tail())

49    1710.0
50    1898.0
51    2941.0
52    2942.0
53    2944.0
Name: cont_classification_task_id, dtype: float64

49    1710.0
50    1898.0
51    2941.0
52    2942.0
53    2944.0
Name: cont_classification_task_id_baseline, dtype: float64

In [15]:
df_matching.shape 

(54, 64)

In [17]:
path = './files/mapping/baseline_image_model_baselineaux_task_mapping.csv'
df_matching.to_csv(path, index=False)

In [18]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [19]:
for i,l1 in tqdm(enumerate(lst)):
    l2 = l1
    print(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'].nunique())

10it [00:00, 138.97it/s]

50
47
28
18
10
6
4
3
2
2


In [21]:
# new approach : for inference, only consider the main tasks that correspond to aux tasks meeting the criteria 
# instead of one general sparse y matrix 

base_path = '../aux_data_preperation/baseline_plus_aux_data/matrices/cls'

for i,l1 in tqdm(enumerate(lst)):
    y_sparse_main_fold0 = lil_matrix(y_aux.shape)
    print(l1)
    l2 = l1
    #for j,l2 in enumerate(lst): 
    file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
    col_idxs = set(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'])
    for col in col_idxs :
        y_sparse_main_fold0[folds == 0,col] = 1
    path = os.path.join(base_path, 'y_sparse_main_tasks_fold0_{}'.format(file_core))
    save_npz(path,csr_matrix(y_sparse_main_fold0))

0it [00:00, ?it/s]

0


1it [00:02,  2.48s/it]

0.2


2it [00:04,  2.41s/it]

...

0.9


9it [00:10,  1.53it/s]

0.95


10it [00:10,  1.08s/it]

0.99
